In [1]:
from matplotlib import pyplot as plt, patches
import numpy as np
from ipywidgets import widgets, interact
from IPython.display import display
import csidata as cid
import pandas as pd
from os.path import basename, splitext, exists

In [2]:
frame_timestamps = None
# Sample data
# data = sio.loadmat('../DeepSeg/01Data_PreProcess/Data_CsiAmplitudeCut/philipp/55philipp_uc_d.mat')['data_']
# data = csidata.load('DeepSeg/01Data_PreProcess/Data_CsiAmplitudeCut/philipp/55philipp_uc_d.mat', csidata.FileFormat.MAT_7_3)
filename = 'recordings/csi/2024-09-27T13-08-02_480164.raw'
filename = 'analysis/outside_recording/2024-09-30T19-55-50_056812_seq-iw.raw'
filename = 'recordings/csi/2024-10-10T17-01-57_429901_ph.raw'
filename = 'recordings/csi/2024-10-11T13-51-55_693628.raw'
filename = 'record/prod_data/csi/user1/2024-10-11T16-53-18-669897_seq-iw.joblib'
# filename = 'recordings/csi/2024-10-15T17-33-03-720294.raw'
# filename = 'recordings/csi/2024-10-16T17-35-42-664895.raw'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-23-58-202160_seq-sd.joblib'
# filename = 'recordings/csi/2024-10-17T17-03-04-151691.raw'

# filename = 'record/prod_data/csi/user1/2024-10-11T17-02-22-296947_seq-ph.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-04-18-741718_seq-ph.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-05-49-170291_seq-ph.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-06-48-096612_seq-ph.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-09-06-000850_seq-ph.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-11-48-279807_seq-ph.joblib'

# filename = 'record/prod_data/csi/user1/2024-10-11T17-14-07-312598_seq-rp.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-16-06-407371_seq-rp.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-17-06-928945_seq-rp.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-18-38-301779_seq-rp.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-19-40-755138_seq-rp.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-20-49-594629_seq-rp.joblib'

# filename = 'record/prod_data/csi/user1/2024-10-11T17-24-50-860221_seq-sd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-27-50-910991_seq-sd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-28-59-804945_seq-sd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-30-24-686743_seq-sd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-31-45-171551_seq-sd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-32-50-910010_seq-sd.joblib'

# filename = 'record/prod_data/csi/user1/2024-10-11T17-46-37-311054_seq-wd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-48-01-357582_seq-wd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-49-22-699238_seq-wd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-50-36-100037_seq-wd.joblib'
# filename = 'record/prod_data/csi/user1/2024-10-11T17-51-49-659091_seq-wd.joblib'
filename = 'record/prod_data/csi/user1/2024-10-11T17-53-07-740347_seq-wd.raw'


label_save_file = 'record/prod_data/user1.csv'

In [3]:
data = cid.load(filename)
csi = data.csi
csi.shape

input_reduction = 20
sel = np.index_exp[::input_reduction, 2, 2, 0]

csiTrace = csi[sel]
csiTrace.shape

(2679,)

In [4]:
def set_csi_trace(rx_idx, tx_idx, subcarrier_idx):
    global csiTrace
    csiTrace = csi[::input_reduction, rx_idx, tx_idx, subcarrier_idx]
    # plot_csi_trace(csiTrace)

In [5]:
import subprocess
import threading

img_thread: threading.Thread = None
img_subprocess: subprocess.Popen = None
prev_frame_idx = -1
video_path = None
image = None
tmp_file = None

def display_video_frame(frame_idx):
    global img_thread, img_subprocess, prev_frame_idx, tmp_file
    # print(f'Requesting frame {frame_idx}')
    if img_thread is not None and img_thread.is_alive():
        if prev_frame_idx == frame_idx:
            print('Same frame being requested, wait for existing thread to finish')
            return
        
        print('Stopping previous subprocess')
        img_subprocess.kill()
    else:
        if prev_frame_idx == frame_idx:
            print('Frame already displayed')
            return
    
    # print('Creating new thread')
    img_thread = threading.Thread(target=get_video_frame, args=(frame_idx,))
    img_thread.start()
    prev_frame_idx = frame_idx


def get_video_frame(frame_idx):
    global image, img_subprocess, tmp_file

    print(f'FFMPEG get frame {frame_idx}')
    img_subprocess = subprocess.Popen(fr'ffmpeg -v error -f rawvideo -pix_fmt yuv422p -s 1280x720 -framerate 60 -i {tmp_file} -vf select=eq(n\,{frame_idx}) -vframes 1 -f image2 -c:v png pipe:'.split(), 
                                  stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    # print('FFMPEG process started')
    out, err = img_subprocess.communicate()
    if img_subprocess.returncode != 0:
        print(f'Error getting frame {frame_idx}')
        print(err)
        return
    
    # print(f'Frame {frame_idx} received')
    
    image.value = out
    # print(f'Set image for frame {frame_idx}')


In [6]:
import datetime

def getNearestIdx(array, value):
    return (np.abs(array - value)).argmin()

def getVideoFrameMatchingCsiPosition(csi_idx: int):
    # limit csi_idx to the range of the csi data
    csi_idx = np.clip(csi_idx, 0, csi.shape[0]-1)
    csi_ts = data.status[::input_reduction.value][csi_idx].tstamp
    print(csi_ts)

    if frame_timestamps is not None:
        frame_idx = getNearestIdx(frame_timestamps, csi_ts)
    else:
        start_time = basename(filename).split('_')[0]
        start_time = datetime.datetime.strptime(start_time, '%Y-%m-%dT%H-%M-%S-%f')

        # video_start_time = start_time + datetime.timedelta(milliseconds=660) # 0.66s delay 
        video_start_time = start_time + datetime.timedelta(milliseconds=660+180) # 0.66s delay 

        csi_dt = datetime.datetime.fromtimestamp(csi_ts/1e6)

        # video frame rate is 60 fps
        # frame_idx = (csi_dt - video_start_time).total_seconds() * 60
        frame_idx = (csi_dt - video_start_time).total_seconds() * 60.35
        max_idx = csi.shape[0] - 1 # TODO get from video
        frame_idx = np.clip(int(frame_idx), 0, max_idx)

    display_video_frame(frame_idx)
    print(frame_idx)

In [7]:
# getVideoFrameMatchingCsiPosition(30, 'record/prod_data/video/user1/2024-10-11T16-53-18-669897_seq-iw.mp4')

In [8]:
class Notifier():
    def __init__(self):
        self.func = []

    def register(self, func):
        self.func.append(func)

    def notify(self):
        for f in self.func:
            f()

In [9]:
from typing import NamedTuple

import pandas as pd

from csidata.types import Activity, CSI_Result

class SegmentListEntry:
    def __init__(self, segment_list_manager, segment_bounds: tuple[int, int], segment_label: str, segment_id: int, segment_type_options: list[tuple[str, int]]):
        self.segment_bounds = segment_bounds
        self.segment_id = segment_id
        self.segment_type = widgets.Dropdown(options=segment_type_options, value=segment_type_options[0][1], layout=widgets.Layout(width='auto'))
        self.item = widgets.HBox()
        self.id_label = widgets.Label(str(segment_id))
        self.label = widgets.Label(segment_label)
        self.segment_list_manager = segment_list_manager

        space = widgets.HTML("<span style='margin-right: 1em;'></span>")

        move_down_button = widgets.Button(description='⬇️', layout=widgets.Layout(width='auto'))
        move_up_button = widgets.Button(description='⬆️', layout=widgets.Layout(width='auto'))
        remove_button = widgets.Button(description='X', layout=widgets.Layout(width='auto'))

        remove_button.on_click(lambda _: self.segment_list_manager.removeSegment(self))
        move_down_button.on_click(lambda _: self.segment_list_manager.move_down(self))
        move_up_button.on_click(lambda _: self.segment_list_manager.move_up(self))

        self.item.children = [self.id_label, space, self.label, self.segment_type, move_down_button, move_up_button, remove_button]

    def update_id(self, new_id: int):
        self.segment_id = new_id
        self.id_label.value = str(new_id)



class Segment(NamedTuple):
    start: int
    end: int
    type: Activity
    

class SegmentListManager:
    def __init__(self, segments: list[Segment], segment_notifier: Notifier, digits_padding: int, csi_result: CSI_Result) -> None:
        self.segment_notifier = segment_notifier
        self.segment_notifier.register(self.update)
        self.segment_list_items: list[SegmentListEntry] = []
        self.segments = segments
        self.csi_result = csi_result
        self.digits_padding = digits_padding
        self.vbox = widgets.VBox(layout=widgets.Layout(width='350px', min_width='300px'))
        
    def update(self) -> None:
        def segment_equals_entry(segment_bounds: tuple[int, int], entry: SegmentListEntry) -> bool:
            return entry.segment_bounds[0] == segment_bounds[0] and entry.segment_bounds[1] == segment_bounds[1]

        if len(self.segments) == len(self.segment_list_items): # two segments have been swapped, update list items
            for idx, s in enumerate(self.segments):
                if not segment_equals_entry(s, self.segment_list_items[idx]):
                    self.segment_list_items[idx].item.close()
                    self.segment_list_items[idx] = self.createSegmentEntry(s, idx)
            
        elif len(self.segments) > len(self.segment_list_items): # segment has been added
            missing_item = [s for s in self.segments if not any(segment_equals_entry(s, e) for e in self.segment_list_items)] [0]
            self.segment_list_items.append(self.createSegmentEntry(missing_item, self.segments.index(missing_item)))

        elif len(self.segments) < len(self.segment_list_items): # segment has been removed
            excess_item = [e for e in self.segment_list_items if not any(segment_equals_entry(s, e) for s in self.segments)] [0]
            self.segment_list_items.remove(excess_item)
            excess_item.item.close()

        # self.segment_list_items = [self.createSegmentEntry(segment) for segment in self.segments]
        self.vbox.children = [e.item for e in self.segment_list_items]
        

    def createSegmentEntry(self, segment_bounds: tuple[int, int], segment_id) -> None:
        label = f"({segment_bounds[0]:>0{self.digits_padding}}, {segment_bounds[1]:>0{self.digits_padding}})"

        allowed_activities: list[Activity]
        if self.csi_result.activity:
            allowed_activities = [self.csi_result.activity]
        elif self.csi_result.sequence:
            allowed_activities = list(self.csi_result.sequence.value)
        else:
            allowed_activities = list(Activity)

        segment_type_options = [(a.name, a.value) for a in allowed_activities]

        return SegmentListEntry(self, segment_bounds, label, segment_id, segment_type_options)

    def removeSegment(self, item) -> None:
        idx = self.segment_list_items.index(item)
        self.segments.pop(idx)
        self.segment_notifier.notify()

    def move_up(self, item) -> None:
        idx = self.segment_list_items.index(item)
        if idx > 0:
            self.segments[idx], self.segments[idx - 1] = self.segments[idx - 1], self.segments[idx]
            self.segment_list_items[idx], self.segment_list_items[idx - 1] = self.segment_list_items[idx - 1], self.segment_list_items[idx]
            self.segment_list_items[idx].update_id(idx)
            self.segment_list_items[idx - 1].update_id(idx - 1)
            self.segment_notifier.notify()

    
    def move_down(self, item) -> None:
        idx = self.segment_list_items.index(item)
        if idx < len(self.segments) - 1:
            self.segments[idx], self.segments[idx + 1] = self.segments[idx + 1], self.segments[idx]
            self.segment_list_items[idx], self.segment_list_items[idx + 1] = self.segment_list_items[idx + 1], self.segment_list_items[idx]
            self.segment_list_items[idx].update_id(idx)
            self.segment_list_items[idx + 1].update_id(idx + 1)
            self.segment_notifier.notify()

    
    def getListBox(self):
        return self.vbox
    
    def save_segments(self):
        segs = []
        for seg in self.segment_list_items:
            segs.append(Segment(seg.segment_bounds[0], seg.segment_bounds[1], Activity(seg.segment_type.value)))
        # print(segs)

        columns=['fileNumber', 'activityNumber', 'startPoint', 'endPoint', 'fileName', 'activityCategory']
        try:
            stored_seg = pd.read_csv(label_save_file)
            print(stored_seg.columns)
        except FileNotFoundError:
            stored_seg = pd.DataFrame(columns=columns)

        file_number = stored_seg['fileNumber'].max() + 1 if not stored_seg.empty else 1 # 1-based indexing
        
        recording_name = splitext(basename(filename))[0]

        # if file already has segments, delete to replace them with the current ones
        stored_seg = stored_seg[stored_seg['fileName'] != recording_name]

        added_segments_list = [(file_number, idx, s.start, s.end, recording_name, sli.segment_type.value) for idx, (s, sli) in enumerate(zip(self.segments, self.segment_list_items))]
        added_segments = pd.DataFrame(added_segments_list, columns=columns)
        print(added_segments)
        added_segments['activityNumber'] += 1 # 1-based indexing
        added_segments['activityCategory'] += 1 # 1-based indexing
        print(added_segments)

        pd.concat([stored_seg, added_segments], ignore_index=True).to_csv(label_save_file, index=False)

In [10]:
class CSIPlot:
    def __init__(self, csiTrace: np.ndarray, segments: list[Segment], segment_notifier: Notifier) -> None:
        self.notifier = segment_notifier
        self.notifier.register(self.redraw_segments)
        self.csiTrace = csiTrace
        self.output = widgets.Output()
        self.segment_selection_start = None
        self.cursorLine = None
        self.segmentStartLine = None

        self.segments = segments

        self.segmentSelectionSpan = None
        self.segmentSpans: list[patches.Polygon] = []
        self.segment_labels: list[plt.Text] = []

        self.margin = 0.1

        # self.background = None
        # self.ln = None

        with self.output:
            with plt.ioff():
                self.fig, self.ax = plt.subplots()
            self.fig: plt.Figure
            self.ax: plt.Axes

            self.fig.canvas.header_visible = False
            # self.fig.canvas.toolbar_visible = 'fade-in-fade-out'
            self.fig.canvas.toolbar_visible = False
            self.fig.canvas.footer_visible = False
            self.fig.canvas.capture_scroll = True
    
            (self.ln,) = self.ax.plot(csiTrace)
            # (self.ln,) = self.ax.plot(csiTrace, animated=True)
            # plt.show(block=False)
            # plt.pause(0.1)

            # self.background = self.fig.canvas.copy_from_bbox(self.fig.bbox)
            # self.ax.draw_artist(self.ln)
            # self.fig.canvas.blit(self.fig.bbox)

            self.cursorLine = self.ax.axvline(0, color='r', alpha=0.5)

            self.segmentStartLine = self.ax.axvline(0, color='g', alpha=0.5)
            self.segmentStartLine.set_visible(False)
            
            self.segmentSelectionSpan = self.ax.axvspan(0, 0, color='orange', alpha=0.2)
            self.segmentSelectionSpan.set_visible(False)
            
        self.fig.canvas.mpl_connect('scroll_event', self.handle_scroll)
            
    def start_selection(self, xpos):
        self.segment_selection_start = xpos
        self.segmentStartLine.set_xdata([xpos])
        self.segmentStartLine.set_visible(True)
        self.segmentSelectionSpan.set_xy([xpos, 0])
        self.segmentSelectionSpan.set_width(0)
        self.segmentSelectionSpan.set_visible(True)
        with self.output:
            self.ax.draw_artist(self.segmentStartLine)
            self.ax.draw_artist(self.segmentSelectionSpan)
            self.fig.canvas.draw_idle()        
    
    def end_selection(self, xpos):
        if not self.segment_selection_start:
            return

        self.segments.append(Segment(self.segment_selection_start, xpos, None))
        self.segment_selection_start = None
        self.segmentStartLine.set_visible(False)
        self.segmentSelectionSpan.set_visible(False)
        with self.output:
            self.redraw_segments()
            self.fig.canvas.draw_idle()        
        self.notifier.notify()
    
    def abort_selection(self):
        self.segment_selection_start = None
        self.segmentStartLine.set_visible(False)
        self.segmentSelectionSpan.set_visible(False)

        with self.output:
            self.fig.canvas.draw_idle()    

    def handle_plot_click(self, xpos):
        if xpos < 0 or xpos >= len(self.csiTrace):
            return
        
        if self.segment_selection_start:
            self.end_selection(xpos)
        else:
            self.start_selection(xpos)

    def draw_cursor(self,xpos):
        self.cursorLine.set_xdata([xpos])
        with self.output:
            self.fig.canvas.draw_idle()

    def draw_current_segment_selection_span(self, cursorPosition):
        if self.segment_selection_start:
            self.segmentSelectionSpan.set_xy([self.segment_selection_start, 0])
            self.segmentSelectionSpan.set_width(cursorPosition - self.segment_selection_start)
            self.ax.draw_artist(self.segmentSelectionSpan)
            with self.output:
                self.fig.canvas.draw_idle()
    def redraw_segments(self):
        for segment in self.segmentSpans:
            segment.remove()
        
        self.segmentSpans.clear()
        
        for segment in self.segments:
            self.segmentSpans.append(self.ax.axvspan(segment.start, segment.end, color='yellow', alpha=0.2))
        
        self.redraw_segment_labels()
        with self.output:
            self.fig.canvas.draw_idle()
    
    def redraw_segment_labels(self):
        for label in self.segment_labels:
            label.remove()

        self.segment_labels.clear()
        visible_yregion = self.ax.get_ylim()
        
        for idx, segment in enumerate(self.segments):
            self.segment_labels.append(self.ax.text((segment.start + segment.end)/2, visible_yregion[1], str(idx), horizontalalignment='center', verticalalignment='top'))

    def redraw(self):
        with self.output:
            self.fig.canvas.draw_idle()

    def move_cursor(self, xpos):
        global video_raw, image, cap, tmp_file
        xpos = int(xpos)
        # display_video_frame(tmp_file, xpos)
        getVideoFrameMatchingCsiPosition(xpos)

        # display_frame_mp4(cap, xpos)
        self.draw_cursor(xpos)
        self.draw_current_segment_selection_span(xpos)
        # self.fig.canvas.flush_events()

    def handle_scroll(self, event):
        if event.inaxes != self.ax: # ignore scroll event outside axis
            return
        zoom_factor = 0.1

        cur_xlim = self.ax.get_xlim()

        xdata = event.xdata

        if event.button == 'up':  # Zoom in
            new_xlim = [xdata - (xdata-cur_xlim[0])*(1-zoom_factor), xdata + (cur_xlim[1]-xdata)*(1-zoom_factor)]
            self.ax.set_xlim(new_xlim)
        elif event.button == 'down':  # Zoom out
            new_xlim = [xdata - (xdata-cur_xlim[0])*(1+zoom_factor), xdata + (cur_xlim[1]-xdata)*(1+zoom_factor)]
            self.ax.set_xlim(new_xlim)

        # limit x range to the data range to avoid out of bounds when determining min/max for this region
        x_idx_in_view = [max(round(new_xlim[0]), 0), 
                         min(round(new_xlim[1]), len(self.csiTrace))]

        new_ylim = [self.csiTrace[x_idx_in_view[0]:x_idx_in_view[1]].min(), self.csiTrace[x_idx_in_view[0]:x_idx_in_view[1]].max()]

        self.ax.set_ylim([new_ylim[0] + new_ylim[0] * self.margin, new_ylim[1] + new_ylim[1] * self.margin])

        self.redraw_segment_labels()
        self.fig.canvas.draw_idle()  # Redraw the figure to update the plot


In [ ]:
%matplotlib widget
segments: list[Segment] = []

segment_notifier = Notifier()
plot = CSIPlot(csiTrace, segments, segment_notifier)
segment_manager = SegmentListManager(segments, segment_notifier, len(str(csiTrace.shape[0])), data)

index_slider = widgets.IntSlider(min=0, max=csiTrace.shape[0] -1, step=10, value=0)
interactiveDings = widgets.interactive(plot.move_cursor, xpos=index_slider)

mark_button = widgets.Button(description='Mark')
unmark_button = widgets.Button(description='Unmark')
mark_button.on_click(lambda _: plot.handle_plot_click(index_slider.value))
unmark_button.on_click(lambda _: plot.abort_selection())

controls_hbox = widgets.HBox([mark_button, unmark_button])

save_button = widgets.Button(description='Save')
save_button.on_click(lambda _: segment_manager.save_segments())
# save_button.layout.width = '100%'    

def handle_segment_click(x):
    global segment_manager

    x_idx = round(x)
    plot.handle_plot_click(x_idx)

plot.fig.canvas.mpl_connect('button_press_event', lambda event: handle_segment_click(round(event.xdata)) if event.inaxes == plot.ax else None)
plot.fig.canvas.mpl_connect('motion_notify_event', lambda event: plot.move_cursor(round(event.xdata)) if event.inaxes == plot.ax else None)

image = widgets.Image()
img_width = 500
image.layout = widgets.Layout(width=f"{img_width}px", height=f"{img_width * 9/16}px", top='50%')

hbox_main_content = widgets.HBox([segment_manager.getListBox(), plot.fig.canvas, image])
hbox_main_content.layout.width = '100%'
plot.fig.canvas.layout.width = 'auto'

# set the hbox border color to red
# hbox_segments_plot.layout.border = '1px solid red'

display(hbox_main_content)


# display(interactiveDings)
display(controls_hbox)
display(save_button)

input_rx = widgets.BoundedIntText(description='rx', value=0, min=0, max=csi.shape[1]-1)
input_tx = widgets.BoundedIntText(description='tx', value=0, min=0, max=csi.shape[2]-1)
input_sc = widgets.BoundedIntText(description='sc', value=0, min=0, max=csi.shape[3]-1)
input_reduction = widgets.BoundedIntText(description='red', value=20, min=1, max=csi.shape[0])
confirm_button = widgets.Button(description='Confirm')

csi_selection = widgets.HBox([input_rx, input_tx, input_sc, input_reduction, confirm_button], layout=widgets.Layout(width='auto'))

def on_confirm_button_clicked(_):
    global csiTrace
    print(f'rx: {input_rx.value}, tx: {input_tx.value}, sc: {input_sc.value}, red: {input_reduction.value}')
    csiTrace = csi[::input_reduction.value, input_rx.value, input_tx.value, input_sc.value]

    plot.ln.remove()

    plot.csiTrace = csiTrace
    # plot.ax.clear()
    (plot.ln,) = plot.ax.plot(csiTrace)
    plot.redraw()
    plot.redraw_segments()
    plot.redraw_segment_labels()

confirm_button.on_click(on_confirm_button_clicked)

display(csi_selection)

# filename = "record/recordings/2024-09-04T13-11-58+02-00.mp4"
filename_stem = splitext(basename(filename))[0]

# video_path = f"recordings/video/{filename_stem}.mp4"
video_path = f"record/prod_data/video/user1/{filename_stem}.mp4"
tmp_folder = "recordings/tmp"
tmp_file = f"{tmp_folder}/{filename_stem}.yuv"

if exists(splitext(video_path)[0] + '.txt'):
    with open(splitext(video_path)[0] + '.txt') as f:
            frames_ts = f.read().splitlines()[1:] # skip first line as it is a comment
            frames_ts = map(int, frames_ts)
    frame_timestamps = np.array(list(frames_ts))
    frame_timestamps = frame_timestamps * 1e3 # convert from ms to us presicion to match csi data
else:
    frame_timestamps = None

# check if tmp video file exists
if not exists(tmp_file):
    # convert video to yuv format
    video_raw =  subprocess.run(f"ffmpeg -y -v error -i {video_path} -f rawvideo -pix_fmt yuv422p -framerate 60 {tmp_file}".split(), stdout=subprocess.PIPE, check=True)
# cap = cv2.VideoCapture('/home/felix/Documents/uni/BaProj/bachelorproject/record/recordings/2024-09-04T13-11-58+02-00.mp4')

# get_video_frame(video_raw.stdout, 0)
getVideoFrameMatchingCsiPosition(0)



Button(description='Save', style=ButtonStyle())

1728661987995028
FFMPEG get frame 0
0


1728662007835150
1162
FFMPEG get frame 1162
1728662003919311
Stopping previous subprocess
925
FFMPEG get frame 925
Error getting frame 1162
b''
1728662002405472
Stopping previous subprocess
834
FFMPEG get frame 834
1728662000466509
Stopping previous subprocess
717
FFMPEG get frame 717
Error getting frame 834
b''
Error getting frame 925
b''
1728662000285472
Stopping previous subprocess
706
FFMPEG get frame 706
Error getting frame 717
b''
1728662000195234
Stopping previous subprocess
700
FFMPEG get frame 700
Error getting frame 706
b''
1728662000285472
Stopping previous subprocess
706
FFMPEG get frame 706
Error getting frame 700
b''
1728662000104383
Stopping previous subprocess
695
FFMPEG get frame 695
1728661999812529
Stopping previous subprocess
677
FFMPEG get frame 677
Error getting frame 695
b''
Error getting frame 706
b''
1728661997779359
Stopping previous subprocess
555
FFMPEG get frame 555
Error getting frame 677
b''
1728661996667595
Stopping previous subprocess
488
FFMPEG get fra

ValueError: <__main__.SegmentListEntry object at 0x7f307d428e60> is not in list